In [ ]:
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas
import pickle
from tqdm import tqdm

In [ ]:
%matplotlib inline

# 1. Preparing and plotting the data

In [ ]:
hackaton_dir = os.path.expanduser('~/qmenta_cnic_1000_brains_challenge')

In [ ]:
volumetric_data_dir = os.path.join(hackaton_dir, 'train')

Get the age of each subject from train.csv

In [ ]:
age_csv_df = pandas.read_csv(os.path.join(hackaton_dir, 'train.csv'), index_col=0)

In [ ]:
age_csv_df.iloc[0].values

In [ ]:
age_csv_df

Get the volumetric data previously downloaded to your disk 

In [ ]:
volumetric_csv_files = sorted(glob.glob(os.path.join(volumetric_data_dir, '*_volumetric.csv')))
len(volumetric_csv_files)

We will use the Gray Matter volume normalized by Intracranial Volume (ICV) as a predictor of age

In [ ]:
x_list = list()
y_list = list()
for csv_file in tqdm(volumetric_csv_files):
    subject_id = int(os.path.basename(csv_file).split('_volumetric.csv')[0])
    age_value = age_csv_df.loc[subject_id].values[0]
    volumetric_df = pandas.read_csv(csv_file)
    gm_volume_value = volumetric_df.loc[volumetric_df['label'] == 'Gray matter', 'volume'].values[0]
    icv_value = volumetric_df.loc[volumetric_df['label'] == 'ICV', 'volume'].values[0]
    x_list.append(gm_volume_value / icv_value)
    y_list.append(age_value)

In [ ]:
x = np.array(x_list, dtype=np.float32)
y = np.array(y_list, dtype=np.float32)

Let's plot the data and see how the ICV and the age are related in a qualitative manner

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(x, y)
plt.xlabel('Normalized Gray Matter volume')
plt.ylabel('Age (years)')

# 2. Linear regression model

It can be seen from the previous plot that the amount of gray matter in the brain correlates linearly with the age. Therefore, we can use a simple linear regression model to predict the age with decent performance in terms of squared error.

We will use scikit-learn to create an Ordinary Least Squares model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_regression_model = LinearRegression(fit_intercept=True, normalize=True)

Make sure the dimensions of X and y are the appropriate ones

In [ ]:
x = x[:, np.newaxis]

In [ ]:
x.shape

In [ ]:
y.shape

Fit the model

In [ ]:
linear_regression_model.fit(x, y)

Get R² fitting score

In [ ]:
linear_regression_model.score(x, y)

Get predictions and coefficients

In [ ]:
y_pred = linear_regression_model.predict(x)

In [ ]:
intercept, coef = linear_regression_model.intercept_, linear_regression_model.coef_[0]
print(intercept, coef)

Plot data and regression line

In [ ]:
x_reg_line = np.linspace(x.min() - 0.01, x.max() + 0.01, 1000)
reg_line = linear_regression_model.predict(x_reg_line[:, np.newaxis])

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(x[:, 0], y, label='Data')
plt.plot(x_reg_line, reg_line, color='green', label='Regression line')
plt.xlabel('Normalized Gray Matter volume')
plt.ylabel('Age (years)')
plt.legend()

Let's plot the residuals also

In [ ]:
residuals = y - y_pred

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(x, residuals)
plt.xlabel('Normalized Gray Matter volume')
plt.ylabel('Residuals (y - ŷ)')

# 3. Persisting the model to disk

We are just going to use **pickle** from Python's standard library to serialize the sklearn instance and dump it to disk.

First we create a folder for our persisted models

In [ ]:
persisted_models_dir = os.path.join(hackaton_dir, 'models')

In [ ]:
os.makedirs(persisted_models_dir, exist_ok=True)

Now we simply invoke `pickle.dump()` to persist the model

In [ ]:
model_path = os.path.join(persisted_models_dir, 'linear_regression_example.pkl')

In [ ]:
with open(model_path, 'wb') as fd:
    pickle.dump(linear_regression_model, file=fd)

Now we can try to load the model from disk and assert that it behaves exactly the same as the in-memory version

In [ ]:
with open(model_path, 'rb') as fd:
    loaded_linear_regression_model = pickle.load(fd)

In [ ]:
y_pred_original = linear_regression_model.predict(x)
y_pred_loaded = loaded_linear_regression_model.predict(x)

In [ ]:
assert np.allclose(y_pred_original, y_pred_loaded)